In [2]:
import tensorflow as tf
import time
# %%time -n1 -r1

In [4]:
tf.__version__

'2.16.0-rc0'

## Prefetch

In [5]:
class FileDataset(tf.data.Dataset):
    def read_file_in_batches(num_samples):
        time.sleep(0.03)
        for sample_idx in range(num_samples):
            time.sleep(0.015)
            
            yield(sample_idx,)
    def __new__(cls, num_samples=3):
        return tf.data.Dataset.from_generator(
            cls.read_file_in_batches, output_signature=tf.TensorSpec(shape=(1,), dtype=tf.int64), args=(num_samples,)
        )

In [6]:
def benchmark(datasets, num_epoch=2):
    for epoch_num in range(num_epoch):
        for sample in datasets:
            time.sleep(0.01)

In [7]:
%%timeit
benchmark(FileDataset())

507 ms ± 30.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
benchmark(FileDataset().prefetch(1))

485 ms ± 74.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

454 ms ± 64.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Cache

In [10]:
dataset = tf.data.Dataset.range(5)
dataset = dataset.map(lambda x: x**2)
dataset = dataset.cache("mycache.txt")
# The first time reading through the data will generate the data using
# `range` and `map`.
list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [12]:

# Subsequent iterations read from the cache.
list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [13]:
def mapped_function(s):
    # Do some hard pre-processing
    tf.py_function(lambda: time.sleep(0.03), [], ())
    return s

In [14]:

%%timeit -r1 -n1
benchmark(FileDataset().map(mapped_function), 5)

1.55 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [15]:

%%timeit -r1 -n1
benchmark(FileDataset().map(mapped_function).cache(), 5)

711 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [1]:
print("hello")

hello
